In [6]:
#Importing...
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Ridge 
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor


In [7]:
#Load data
data = pd.read_csv('cleaned_data.csv')

data.head()

,Unnamed: 0,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,0,19,27.900,0,16884.92400,0,1,0,0,1
1,1,18,33.770,1,1725.55230,1,0,0,1,0
2,2,28,33.000,3,4449.46200,1,0,0,1,0
3,3,33,22.705,0,21984.47061,1,0,1,0,0
4,4,32,28.880,0,3866.85520,1,0,1,0,0


In [8]:
#Features and target variables

X = data.drop(columns=['charges'])
y = data['charges']

In [9]:
#Split data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [10]:
#Models

linerarRegressor = LinearRegression()
linerarRegressor.fit(X_train, y_train)
predictions_lr = linerarRegressor.predict(X_test)


randomForestModel = RandomForestRegressor(n_estimators=100, random_state=42)
randomForestModel.fit(X_train, y_train)
predictions_rf = randomForestModel.predict(X_test)


In [11]:
#Model Evaluation
mse_lr = mean_squared_error(y_test, y_pred=predictions_lr)

r2_lr= r2_score(y_test, y_pred=predictions_lr)

print(f"\nMean Squared Error for Linear Regression : {mse_lr}\n"
      f"\nR-squared Score Linear Regression: {r2_lr}"
      )

mse_rf = mean_squared_error(y_test, y_pred=predictions_rf)

r2_rf= r2_score(y_test, y_pred=predictions_rf)

print(f"\nMean Squared Error of Random Forest: {mse_rf}\n"
      f"\nR-squared Score of Random Forest: {r2_rf}"
      )


Mean Squared Error for Linear Regression : 35564984.47965559

R-squared Score Linear Regression: 0.8064554513330181

Mean Squared Error of Random Forest: 20478073.23579169

R-squared Score of Random Forest: 0.8885583812286515


In [12]:

#Standardization
scaler_std = StandardScaler()
X_train_std = scaler_std.fit_transform(X_train)
X_test_std = scaler_std.fit_transform(X_test)

#Normalization
scaler_minmax = MinMaxScaler()
X_train_minmax = scaler_minmax.fit_transform(X_train)
X_test_minmax = scaler_minmax.fit_transform(X_test)


In [13]:

ridge = Ridge(alpha=1.0)
ridge.fit(X_train_std, y_train)

predictions_ridge = ridge.predict(X_test_std)

mse_ridge = mean_squared_error(y_test, predictions_ridge)
r2_ridge = r2_score(y_test, predictions_ridge)

print("\nRidge Regression Mean Squared Error:\n", mse_ridge)
print("\nRidge Regression R2 score:\n", r2_ridge)


Ridge Regression Mean Squared Error:
 38366284.09655684

Ridge Regression R2 score:
 0.7912107864479742


Ridge Regression:
Ridge Regression is a type of regularized linear regression that can help in handling multicollinearity and prevent overfitting by adding a regularization term to the cost function

In [14]:

#Set up the pipeline with scaling and the Ridge model
steps = [
    ('scaler', StandardScaler()),
    ('model', Ridge()) 
]

pipeline = Pipeline(steps)

#Hyperparameters grid to search over
param_grid = {
    'model__alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]  # Values of alpha to test
}

# Perform GridSearchCV
grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(X_train, y_train)

#Best hyperparameters and best score
print("\nBest Hyperparameters:\n", grid.best_params_)
print("\nBest Cross-Validation MSE:\n", -grid.best_score_)

#Evaluate on test set using best model
best_model = grid.best_estimator_
test_predictions = best_model.predict(X_test)

test_mse = mean_squared_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)

print("\nTest Set MSE with Best Model:\n", test_mse)
print("\nTest Set R2 with Best Model:\n", test_r2)


Best Hyperparameters:
 {'model__alpha': 10.0}

Best Cross-Validation MSE:
 37629448.590384945

Test Set MSE with Best Model:
 35915547.839589596

Test Set R2 with Best Model:
 0.8045476864831158


XGBoost: Gradient Boosting Regression (e.g., XGBoost, LightGBM):
Model: Builds trees sequentially, focusing on areas where previous trees performed poorly. We will aso tune the hyperparameters to optimize model


In [15]:
xgb = XGBRegressor()

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5]
}
grid = GridSearchCV(xgb, param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(X_train, y_train)

#Best parameters
print("\nBest Hyperparameters:\n", grid.best_params_)
print("\nBest Cross-Validation MSE:\n", -grid.best_score_)

#Model Evaluation
best_model = grid.best_estimator_
test_predictions = best_model.predict(X_test)

test_mse_xgb = mean_squared_error(y_test, test_predictions)
test_r2_xgb = r2_score(y_test, test_predictions)

print("\nTest Set MSE with Best Model:\n", test_mse_xgb)
print("\nTest Set R2 with Best Model: \n", test_r2_xgb)



Best Hyperparameters:
 {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300}

Best Cross-Validation MSE:
 21658101.807436105

Test Set MSE with Best Model:
 19114153.728462704

Test Set R2 with Best Model: 
 0.895980827472515


In [16]:
Model = tf.keras.Sequential([
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1)
]
)
Model.compile(
    loss = tf.keras.losses.mae,
    optimizer = tf.keras.optimizers.legacy.Adam(lr = 0.01),
    metrics = ['mae']
)
Model.fit(X_train,y_train,epochs= 200)

Epoch 1/200
 1/34 [..............................] - ETA: 5s - loss: 13809.3906 - mae: 13809.3906

/Users/bisialgoruthm/Documents/Python/Predictive-Modeling-of-Health-Insurance-Charges--Understanding-Factors-Influencing-Costs-/.venv/lib/python3.9/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


34/34 [==============================] - 0s 639us/step - loss: 9893.6846 - mae: 9893.6846
Epoch 2/200
34/34 [==============================] - 0s 546us/step - loss: 8796.6914 - mae: 8796.6914
Epoch 3/200
34/34 [==============================] - 0s 501us/step - loss: 8071.3096 - mae: 8071.3096
Epoch 4/200
34/34 [==============================] - 0s 525us/step - loss: 7192.1934 - mae: 7192.1934
Epoch 5/200
34/34 [==============================] - 0s 518us/step - loss: 6991.5747 - mae: 6991.5747
Epoch 6/200
34/34 [==============================] - 0s 567us/step - loss: 6873.4766 - mae: 6873.4766
Epoch 7/200
34/34 [==============================] - 0s 552us/step - loss: 6692.8271 - mae: 6692.8271
Epoch 8/200
34/34 [==============================] - 0s 529us/step - loss: 6588.8188 - mae: 6588.8188
Epoch 9/200
34/34 [==============================] - 0s 494us/step - loss: 6451.8389 - mae: 6451.8389
Epoch 10/200
34/34 [==============================] - 0s 526us/step - loss: 6372.8613 - mae: 6

In [17]:

plot_model(Model,show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [18]:
ev = Model.evaluate(X_test,y_test)

9/9 [==============================] - 0s 550us/step - loss: 4033.3828 - mae: 4033.3828


In [19]:
tf.random.set_seed(42)
model_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'relu'),
])

model_1.compile(loss=tf.keras.losses.mae,
                optimizer = tf.keras.optimizers.Adam(lr = 0.001), 
                metrics = ['mae'])

model_1.fit(X_train, y_train, batch_size = 32, epochs = 300)

Epoch 1/300
34/34 [==============================] - 0s 795us/step - loss: 12878.5391 - mae: 12878.5391
Epoch 2/300
34/34 [==============================] - 0s 716us/step - loss: 12264.0645 - mae: 12264.0645
Epoch 3/300
34/34 [==============================] - 0s 612us/step - loss: 10636.3477 - mae: 10636.3477
Epoch 4/300
34/34 [==============================] - 0s 471us/step - loss: 9218.1475 - mae: 9218.1475
Epoch 5/300
34/34 [==============================] - 0s 438us/step - loss: 9065.5811 - mae: 9065.5811
Epoch 6/300
34/34 [==============================] - 0s 455us/step - loss: 8986.7412 - mae: 8986.7412
Epoch 7/300
34/34 [==============================] - 0s 464us/step - loss: 8904.4805 - mae: 8904.4805
Epoch 8/300
34/34 [==============================] - 0s 491us/step - loss: 8813.7832 - mae: 8813.7832
Epoch 9/300
34/34 [==============================] - 0s 471us/step - loss: 8713.0410 - mae: 8713.0410
Epoch 10/300
34/34 [==============================] - 0s 471us/step - loss: 

In [20]:
ev_1 = model_1.evaluate(X_test, y_test)
ev_1

9/9 [==============================] - 0s 660us/step - loss: 3752.9453 - mae: 3752.9453


[3752.9453125, 3752.9453125]

Lets add another layer:

In [21]:
tf.random.set_seed(42)
model_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'relu')
])

model_2.compile(loss=tf.keras.losses.mae,
                optimizer = tf.keras.optimizers.Adam(lr = 0.001),
                metrics = ['mae'])

model_2.fit(X_train, y_train, batch_size = 32, epochs = 500)

Epoch 1/500
34/34 [==============================] - 0s 772us/step - loss: 12376.6963 - mae: 12376.6963
Epoch 2/500
34/34 [==============================] - 0s 666us/step - loss: 9808.5332 - mae: 9808.5332
Epoch 3/500
34/34 [==============================] - 0s 615us/step - loss: 9099.5967 - mae: 9099.5967
Epoch 4/500
34/34 [==============================] - 0s 538us/step - loss: 8888.1123 - mae: 8888.1123
Epoch 5/500
34/34 [==============================] - 0s 505us/step - loss: 8574.0176 - mae: 8574.0176
Epoch 6/500
34/34 [==============================] - 0s 523us/step - loss: 8077.7354 - mae: 8077.7354
Epoch 7/500
34/34 [==============================] - 0s 523us/step - loss: 7542.1401 - mae: 7542.1401
Epoch 8/500
34/34 [==============================] - 0s 529us/step - loss: 7234.2168 - mae: 7234.2168
Epoch 9/500
34/34 [==============================] - 0s 580us/step - loss: 7164.7275 - mae: 7164.7275
Epoch 10/500
34/34 [==============================] - 0s 534us/step - loss: 7212

In [22]:
ev_2 = model_2.evaluate(X_test, y_test)

9/9 [==============================] - 0s 580us/step - loss: 2483.9810 - mae: 2483.9810


In [23]:
model_3 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1)
])

model_3.compile(loss = tf.keras.losses.mae,
               optimizer = tf.keras.optimizers.Adam(lr=0.01),
               metrics = ['mae'])

model_3.fit(X_train, y_train, batch_size = 32, epochs = 1000)

Epoch 1/1000
34/34 [==============================] - 0s 873us/step - loss: 12182.0361 - mae: 12182.0361
Epoch 2/1000
34/34 [==============================] - 0s 778us/step - loss: 9299.2842 - mae: 9299.2842
Epoch 3/1000
34/34 [==============================] - 0s 740us/step - loss: 8994.8623 - mae: 8994.8623
Epoch 4/1000
34/34 [==============================] - 0s 627us/step - loss: 8499.8086 - mae: 8499.8086
Epoch 5/1000
34/34 [==============================] - 0s 650us/step - loss: 7498.8735 - mae: 7498.8735
Epoch 6/1000
34/34 [==============================] - 0s 633us/step - loss: 7178.1445 - mae: 7178.1445
Epoch 7/1000
34/34 [==============================] - 0s 655us/step - loss: 7073.9712 - mae: 7073.9712
Epoch 8/1000
34/34 [==============================] - 0s 641us/step - loss: 7067.8315 - mae: 7067.8315
Epoch 9/1000
34/34 [==============================] - 0s 674us/step - loss: 6923.7510 - mae: 6923.7510
Epoch 10/1000
34/34 [==============================] - 0s 644us/step - 